### 네이버 맛집 테이블

* 1. 맛집리스트 (식당이름(naver_keyword), 주소, 영업시간, 평점, 리뷰갯수, 전화번호)
* 2. 맛집리뷰 (사용자id, 리뷰내용, 작성일자, 평점)

In [32]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import glob

import re

In [3]:
input_path = "*202206.csv"
file_list = [f for f in glob.glob(input_path)]
matzip=pd.DataFrame()

for file in file_list:
    temp = pd.read_csv(file,encoding='utf-8',usecols =['상가업소번호','상호명', '상권업종대분류명','상권업종중분류명',
                                '상권업종소분류명','도로명','경도', '위도'])
    matzip=matzip.append(temp)

matzip.columns=['store_id','store_name','cate_a','cate_b','cate_c','addr','lon','lat']

matzip=matzip[matzip['cate_a']=="음식"]
matzip=matzip[matzip['cate_b']!="제과제빵떡케익"]
matzip=matzip[matzip['cate_b']!="음식배달서비스"]
matzip=matzip[matzip['cate_b']!="유흥주점"]


matzip.reset_index(inplace=True)
matzip.drop(['index'],axis=1,inplace=True)
matzip.drop(['cate_a'],axis=1,inplace=True)

matzip['naver_keyword'] = matzip['store_name'] + " " + matzip['addr']
matzip.to_csv("matzip_naver.csv", index = False)

C:\Users\tj\AppData\Local\Temp\ipykernel_6776\2004451417.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matzip=matzip.append(temp)
C:\Users\tj\AppData\Local\Temp\ipykernel_6776\2004451417.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matzip=matzip.append(temp)
C:\Users\tj\AppData\Local\Temp\ipykernel_6776\2004451417.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matzip=matzip.append(temp)


In [176]:
matzip=pd.read_csv('matzip_naver.csv')

In [177]:
# 연습용으로 50개만 돌림
matzip=matzip[:2]

In [178]:
matzip

,store_id,store_name,cate_b,cate_c,addr,lon,lat,naver_keyword
0,20544673,큰손의정부부대찌개,한식,부대찌개/섞어찌개,경기도 김포시 감정로,126.699974,37.62059,큰손의정부부대찌개 경기도 김포시 감정로
1,23323139,앞족애,한식,한식/백반/한정식,경기도 의정부시 둔야로,127.039861,37.73933,앞족애 경기도 의정부시 둔야로


In [179]:
def text_processing(text): # 한글 영어만 나오도록 설정(예:ㅋㅋㅋ 가능, ㅋㅋㅋ 뺄 거면 ㄱ-ㅣ 삭제하면 됨)
    special = re.compile(r'[^ A-Z a-z 0-9 ㄱ-ㅣ가-힣+]')
    result = special.sub('',text)
    return result

In [180]:
def date_processing(text): # 날짜전처리
    special = re.compile(r'[^ 0-9 . ]')
    result = special.sub('',text[:-6])
    
    if result.count('.')==2:
        result='2023.'+result
    else:
        result='20'+result
    return result

In [173]:
chromedriver = '\\Program Files\\Google\\Chrome\\Application\\chrome.exe'
driver = webdriver.Chrome(chromedriver)

for key in matzip['naver_keyword'].tolist():
    
    naver_map_url = f"https://map.naver.com/v5/search"
    driver.get(naver_map_url)
    driver.implicitly_wait(5)
        
    driver.switch_to.frame('entryIframe')
    # Iframe으로 전환시 기다리지 않으면 파싱되지않음
    driver.implicitly_wait(5)
    
    search = driver.find_element_by_id('input_search1675392885271')
    search.send_keys(key)
    
    button = driver.find_element_by_class_name('button_search')
    button.click()
    

C:\Users\TJ\AppData\Local\Temp\ipykernel_11080\2556556648.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


NoSuchFrameException: Message: entryIframe


### 네이버 맛집 크롤링

In [198]:
chromedriver = '\\Program Files\\Google\\Chrome\\Application\\chrome.exe'
driver = webdriver.Chrome(chromedriver)

stores_info_list=[]
all_mem_review_list=[]

for i, keyword in enumerate(matzip['naver_keyword'].tolist()):
    
        naver_map_url = f"https://map.naver.com/v5/search/{keyword}"
        driver.get(naver_map_url)
        driver.implicitly_wait(5)
        
        driver.switch_to.frame('entryIframe')
        # Iframe으로 전환시 기다리지 않으면 파싱되지않음
        driver.implicitly_wait(5)
        
        # 아직까지도 운영하는 식당이라면
        if (len(driver.find_element(By.CLASS_NAME,'Fc1rA').text) > 0):
            
            ##########상가업소번호
            store_id = matzip['store_id'][i]

            ##########리뷰
            try:
                star = driver.find_element(By.CLASS_NAME,'PXMot.LXIwF').text
                star_point= star[star.find('\n')+1:star.find('/')]

            except:
                star_point = 0.0

            ##########리뷰갯수
            try:
                star_count = driver.find_element(By.CLASS_NAME,'place_bluelink').text

            except:
                star_count = 0
                
                
            try:
                driver.find_element(By.XPATH, "//span[text()='리뷰']").click()
                driver.implicitly_wait(10)

                # 리뷰 더보기가 없을때까지 무한 클릭
#                 while True:
                
                try:  
                    btn_more = driver.find_element(By.CLASS_NAME,'fvwqf')       
                    btn_more.click()

                except: 
                    break

                # 리뷰더보기 모두 클릭
                for button in driver.find_elements(By.CSS_SELECTOR,'a.xHaT3'):
                    try:
                        button.click()
                    except:
                        pass

                html = driver.page_source 
                soup = BeautifulSoup(html, 'html.parser')

                time.sleep(10)

                review_list = soup.find_all('li', attrs = {'class':'YeINN'})

#                 for review in review_list:
# # #                     try:
# #                         # 리뷰아이디
# #                         nickname = review.find('div', attrs = {'class':'sBWyy'}).get_text()
# #                         # 리뷰아이디 특수문자제거
# #                         nickname = text_processing(nickname)

# #                         # 리뷰내용
# #                         review_content = review.find('span', attrs = {'class':'zPfVt'}).get_text()
# #                         # 리뷰내용 특수문자제거
# #                         review_content = text_processing(review_content)
# # #                         print(review_content)
# #                         # 개인리뷰평점
# # #                         review_date = review.select_one('div.sb8UA').get_text().split(' ')[1]
#                         try:
# #                             review_star = review.select('div.sb8UA > span.P1zUJ.HNG_1 > span.place_blind')
#                             review_star = driver.find_element(By.CLASS_NAME,'P1zUJ.HNG_1').text
#                             print(review_star)
                            
# #                             review_date = review.find_element(By.XPATH, "//div[@class='sb8UA'] > span[@class='P1zUJ'] > span[text()='년']")
# # #                             review_date = review.select('div.sb8UA > span.P1zUJ > span.place_blind ').get_text()
#                             review_date = date_processing(review_date)
#                         except:
#                             review_star = "리뷰정보없음"
#                             print(review_star)
                                                        
# #                         mem_list=[store_id,nickname,review_date,review_content,review_star]
# #                         all_mem_review_list.append(mem_list)

                        time.sleep(1)

#                     except:
#                         # 리뷰아이디
#                         nickname = review.find('div', attrs = {'class':'sBWyy'}).get_text()
#                         nickname = text_processing(nickname)
                        
#                         # 리뷰내용
#                         review_content = review.find('span', attrs = {'class':'zPfVt'}).get_text()
#                         review_content = text_processing(review_content)

#                         # 개인리뷰평점
#                         review_star = review.select_one('div.sb8UA').get_text().split(' ')[0]

#                         # 리뷰날짜
#                         review_date = review.select_one('div.sb8UA').get_text().split(' ')[1]

#                         mem_list=[store_id,nickname,review_date,review_content,review_star]
#                         all_mem_review_list.append(mem_list)

#                         time.sleep(1)

            except:
                # 리뷰버튼이 없으면..
                # 리뷰가 없다면..
                pass
                
        else:
            # 운영하지않는식당임
            pass
        
        store_info_list=[store_id,star_point,star_count,naver_map_url]
        stores_info_list.append(store_info_list)
            
driver.quit()

C:\Users\TJ\AppData\Local\Temp\ipykernel_11080\364115715.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5
별점
5


In [194]:
mem_list

[20544673, 'lym', '2020.3.13.', '굿이요', '리뷰정보없음']

In [195]:
all_mem_review_list

[[20544673,
  '맛없는건정말못참아',
  '2020.3.13.',
  '친절하세요 무난한 맛이고 라면이 유료라서 아쉬워요',
  '리뷰정보없음'],
 [20544673, '냥냥3226', '2020.3.13.', '좋아요', '리뷰정보없음'],
 [20544673, 'sap', '2020.3.13.', '제 입맛에 맛있어요', '리뷰정보없음'],
 [20544673, '사실은 땅따리', '2020.3.13.', '좋아요', '리뷰정보없음'],
 [20544673, '냥냥3226', '2020.3.13.', '좋아요', '리뷰정보없음'],
 [20544673, '어여쁜수니', '2020.3.13.', '늘 가는곳 맛있어요 사장님도 친절하세요', '리뷰정보없음'],
 [20544673, '내동생일산치킨살인마', '2020.3.13.', '굿', '리뷰정보없음'],
 [20544673, '냥냥3226', '2020.3.13.', '너무 맛있어요', '리뷰정보없음'],
 [20544673, '사실은 땅따리', '2020.3.13.', '너무너무 맛있구여사장님이 너무 친절하세요', '리뷰정보없음'],
 [20544673, '빠담빠담40', '2020.3.13.', '굿', '리뷰정보없음'],
 [20544673, '민자9', '2020.3.13.', '굿', '리뷰정보없음'],
 [20544673, 'jhj276', '2020.3.13.', '맛이 없었어요양은 좀 많으나', '리뷰정보없음'],
 [20544673, '냥냥3226', '2020.3.13.', '맛있어요', '리뷰정보없음'],
 [20544673, '다섯손가락3263', '2020.3.13.', '김치가 맛있어요', '리뷰정보없음'],
 [20544673, 'nae', '2020.3.13.', '맛있어요', '리뷰정보없음'],
 [20544673,
  '삼나무가로수길',
  '2020.3.13.',
  '맛은 좋은데 양이 항상 문제네요 2인분 먹다가 3인분 시켜봤는데 2인분이랑 내용물이 똑같아요

In [182]:
stores_info_list

[[20544673,
  '4.32',
  '방문자리뷰 90',
  'https://map.naver.com/v5/search/큰손의정부부대찌개 경기도 김포시 감정로'],
 [23323139, 0.0, 0, 'https://map.naver.com/v5/search/앞족애 경기도 의정부시 둔야로']]

### 맛집 리스트 전처리

In [140]:
stores_info_df = pd.DataFrame(
    stores_info_list, columns=['store_id','naver_star_avg','naver_review_num','naver_url']
)

In [141]:
stores_info_df.head(2)

,store_id,naver_star_avg,naver_review_num,naver_url
0,20544673,4.32,방문자리뷰 90,https://map.naver.com/v5/search/큰손의정부부대찌개 경기도 ...
1,23323139,0.0,0,https://map.naver.com/v5/search/앞족애 경기도 의정부시 둔야로


In [142]:
# 전처리
stores_info_df['naver_review_num'] = stores_info_df['naver_review_num'].str.replace("방문자리뷰","")
stores_info_df['naver_review_num'] = stores_info_df['naver_review_num'].fillna(0)

In [143]:
stores_info_df = pd.merge(
    stores_info_df, matzip,
    how = 'left', on = 'store_id'
)

In [144]:
stores_info_df.head(1)

,store_id,naver_star_avg,naver_review_num,naver_url,store_name,cate_b,cate_c,addr,lon,lat,naver_keyword
0,20544673,4.32,90,https://map.naver.com/v5/search/큰손의정부부대찌개 경기도 ...,큰손의정부부대찌개,한식,부대찌개/섞어찌개,경기도 김포시 감정로,126.699974,37.62059,큰손의정부부대찌개 경기도 김포시 감정로


In [16]:
stores_info_df.to_csv("naver_stores_info_df.csv", encoding = 'cp949', index = False)

### 맛집 리뷰내용 전처리

In [145]:
review_info_df = pd.DataFrame(
    all_mem_review_list, columns=['store_id','naver_nickname','naver_date','naver_content','naver_star']
    )

In [146]:
review_info_df.head(1)

,store_id,naver_nickname,naver_date,naver_content,naver_star
0,20544673,맛없는건정말못참아,2023.1.8.,친절하세요 무난한 맛이고 라면이 유료라서 아쉬워요,최근


In [147]:
# 닉네임이널값인행제거
review_info_df['naver_nickname'] = review_info_df['naver_nickname'].dropna(axis = 0)

In [148]:
review_info_df['naver_star'] = review_info_df['naver_star'].str.replace('별점','')
review_info_df['naver_star'] = review_info_df['naver_star'].str.replace('최근','')

In [149]:
review_info_df

,store_id,naver_nickname,naver_date,naver_content,naver_star
0,20544673,맛없는건정말못참아,2023.1.8.,친절하세요 무난한 맛이고 라면이 유료라서 아쉬워요,
1,20544673,냥냥3226,2023.1.27.,좋아요,
2,20544673,sap,2022.8.23.,제 입맛에 맛있어요,
3,20544673,사실은 땅따리,2022.8.9.,좋아요,
4,20544673,냥냥3226,2022.1.9.,좋아요,
5,20544673,어여쁜수니,2021.10.24.,늘 가는곳 맛있어요 사장님도 친절하세요,5
6,20544673,내동생일산치킨살인마,2021.12.18.,굿,
7,20544673,냥냥3226,2021.9.18.,너무 맛있어요,5
8,20544673,사실은 땅따리,2022.1.9.,너무너무 맛있구여사장님이 너무 친절하세요,
9,20544673,빠담빠담40,2021.11.20.,굿,


In [132]:
review_info_df.to_csv("naver_review_info_df.csv", encoding = 'cp949', index = False)